# The Battle of Neighborhoods - Final Report (Week 1 and 2) 

### 1. Introduction: COFFEE ETHNIC IN DA NANG

In such a large and rich of coffee culture city like Da Nang, Viet Nam, it will be competitive to start up coffee business. In this case my contractor is a humble Vietnamese man who has contacted me to give advises and draw up essential lines of business prediction and back-up plans (but this part we will just discuss about predicting hot spot).

### 2. Orientation

First of all we need to collect Data of all coffee shops in Da Nang including their name, id, location (address, latitude, longitude) then pick up the "hot" neighbor where locates most of the venue. In order to asset Data we use FourSquare and apply folium for visualizing a particular neighbor in which that we will observe customer "traffic" and predict an appropriate location of new coffee shop in town. In this case you will find its temporary name on the folium map, "O Day Roi!"(Meaning "Here It Is!" in Vietnamese)

Upload Libraries Required

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

### Apply your credential ID on [FourSquare]

In [31]:
CLIENT_ID = 'FTCOBENCC2XHE5IKD4OAVWYWYPCZGJSAXZNAZRS2N4QRM5QR' # your Foursquare ID
CLIENT_SECRET = 'ZRBEZHK35IHDA2VLSPLPVAGOPJTNCLZM2D3XPG0IH0X5BD1N' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FTCOBENCC2XHE5IKD4OAVWYWYPCZGJSAXZNAZRS2N4QRM5QR
CLIENT_SECRET:ZRBEZHK35IHDA2VLSPLPVAGOPJTNCLZM2D3XPG0IH0X5BD1N


##### Get requests near Da Nang city.

In [32]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Da Nang",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)


##### Transform data into json then request geocode.

In [33]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [40]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Da Nang', 'Da Nang')

In [41]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 16.076950003892705, 'lng': 108.22982675100442},
  'sw': {'lat': 16.059727142671775, 'lng': 108.21333222890732}},
 40)

In [42]:
d["geocode"]

{'what': '',
 'where': 'da nang',
 'center': {'lat': 16.06778, 'lng': 108.22083},
 'displayString': 'Da Nang, Thành Phố Đà Nẵng, Vietnam',
 'cc': 'VN',
 'geometry': {'bounds': {'ne': {'lat': 16.11072, 'lng': 108.276871},
   'sw': {'lat': 15.982205, 'lng': 108.141861}}},
 'slug': 'turan-vietnam',
 'longId': '72057594039511928'}

##### We will start creating group including information which is recommended.

In [43]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [44]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

#### Creating items of objects coffee shop and their attributes - id, address, name, etc

In [45]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 40


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a26a41a31ac6c676705e94c',
  'name': 'cộng cà phê',
  'location': {'address': '39',
   'crossStreet': 'Nguyễn Thái Học',
   'lat': 16.068062879455486,
   'lng': 108.22351222071603,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.068062879455486,
     'lng': 108.22351222071603}],
   'postalCode': '551105',
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['39 (Nguyễn Thái Học)',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

### Testing by calling an item

From the output we can identify necessary factors of what we will use later to consider the probability of launching our up-to-coming location.

Based on that we start to organize what we have got.

In [47]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '39', 'crossStreet': 'Nguyễn Thái Học', 'lat': 16.068062879455486, 'lng': 108.22351222071603, 'labeledLatLngs': [{'label': 'display', 'lat': 16.068062879455486, 'lng': 108.22351222071603}], 'postalCode': '551105', 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['39 (Nguyễn Thái Học)', 'Đà Nẵng', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '17 Phan Đình Phùng', 'lat': 16.070069923328905, 'lng': 108.2234060288238, 'labeledLatLngs': [{'label': 'display', 'lat': 16.070069923328905, 'lng': 108.2234060288238}], 'cc': 'VN', 'city': 'Hải Châu', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['17 Phan Đình Phùng', 'Hải Châu', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '96-98 Bạch Đằng', 'lat': 16.069014884488677, 'lng': 108.22497088340684, 'labeledLatLngs': [{'label': 'display', 'lat': 16.069014884488677, 'lng': 108.22497088340684}], 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'c

,uid,name,shortname,address,postalcode,lat,lng
0,5a26a41a31ac6c676705e94c,cộng cà phê,Coffee Shop,,551105,16.068063,108.223512
1,5627c637498e21ca8f40f4be,Leaf Drinks and More,Coffee Shop,,550000,16.067020,108.224790
2,55ff7cf0498e203b2932d5fd,Namunamu Cafe 180,Café,,550000,16.066268,108.224496
3,51bd1671498e58791c2ea92d,Freeman Coffee,Coffee Shop,,550000,16.070851,108.221230
4,4c25fdd8f1272d7f33ac85c5,Highlands Coffee,Café,,60999,16.069946,108.224961


In [48]:
df.head()

,uid,name,shortname,address,postalcode,lat,lng
0,5a26a41a31ac6c676705e94c,cộng cà phê,Coffee Shop,,551105,16.068063,108.223512
1,5627c637498e21ca8f40f4be,Leaf Drinks and More,Coffee Shop,,550000,16.067020,108.224790
2,55ff7cf0498e203b2932d5fd,Namunamu Cafe 180,Café,,550000,16.066268,108.224496
3,51bd1671498e58791c2ea92d,Freeman Coffee,Coffee Shop,,550000,16.070851,108.221230
4,4c25fdd8f1272d7f33ac85c5,Highlands Coffee,Café,,60999,16.069946,108.224961


##### As we can see that there are many coffee shops without address we need to execute hasattr() to determine if each object (coffeeshop) has a attribute (address).
##### Next step we will execute a very important part - get coordinates of Da Nang and create folium map which will help visualize what we have got from data.

In [49]:
DaNang_center = d["geocode"]["center"]
DaNang_center

{'lat': 16.06778, 'lng': 108.22083}

In [50]:
from folium import plugins


map_DaNang = folium.Map(location=[16.06778, 108.22083], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_DaNang)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_DaNang.add_child(plugins.HeatMap(hm_data))

map_DaNang

### By spotting the clusters of items we can see which neighborhood has density of coffee business.

## 3. Conclusion

#### We will need a location where we can catch out customers from "hot" location we have picked up from the map and stay in a certain distance so as to lessen the competivity of business.

In [51]:
lat = 16.06778
lng = 108.22083
map_DaNang = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_DaNang)
map_DaNang

### Look at the blue buble, here we find out that it will locate about the crossroad between Pham Hong Thai street and Nguyen Chi Thanh street. As I know this neighborhood is safe and right at the center, it's off to the hot spot of night life and on the passing-by path of high school students in the area.